playing around with mmlu pro https://huggingface.co/datasets/TIGER-Lab/MMLU-Pro

In [1]:
%pip install dspy-ai
%pip install ipywidgets
%pip install IPython
%pip install requests

You should consider upgrading via the '/Users/raymondweitekamp/.pyenv/versions/3.10.0/envs/my-env310/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/raymondweitekamp/.pyenv/versions/3.10.0/envs/my-env310/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/raymondweitekamp/.pyenv/versions/3.10.0/envs/my-env310/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/raymondweitekamp/.pyenv/versions/3.10.0/envs/my-env310/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import dspy
from dspy.datasets import DataLoader
import dspy
import os
import dotenv
import requests
import pydantic

setup DSPy

In [3]:
dotenv.load_dotenv()
assert 'OPENAI_API_KEY' in os.environ
llm = dspy.OpenAI(model='gpt-4o', max_tokens=4096, temperature=0.1)
dspy.settings.configure(lm=llm)

Download MMLU-Pro

In [4]:
dl = DataLoader()
mmlu_pro = dl.from_huggingface("TIGER-Lab/MMLU-Pro")
validation = mmlu_pro['validation']
#display(validation)



Get 10 random entries and format for DSPy 

In [7]:
import random

try10 = random.sample(validation, 10)
#display(try10)

try10ex = [example.with_inputs("question", "options") for example in random.sample(validation, 10)]
for ex in try10ex:
    ex['options'] = str(ex['options'])
    ex['answer_index'] = str(ex['answer_index'])
display(try10ex)


[Example({'question_id': 9, 'question': 'Why are parvoviruses a highly impactful parasite?', 'options': '["They are able to alter the host\'s DNA", \'Because they have no nucleic acid\', \'They can survive in extreme temperatures\', \'Only replicate in dividing cells\', \'They can infect multiple species\', "They don\'t require a host to survive", \'Can integrate into host chromosomes\', \'N/A\', \'N/A\', \'N/A\']', 'answer': 'B', 'answer_index': '1', 'cot_content': "A: Let's think step by step. We refer to Wikipedia articles on virology for help. Paroviruses are highly impactful because they do not have nucleic acid. The answer is (B).", 'category': 'health', 'src': 'cot_lib-virology'}) (input_keys={'options', 'question'}),
 Example({'question_id': 11, 'question': 'Where do most short-period comets come from and how do we know?', 'options': "['The Kuiper belt; short period comets tend to be in the plane of the solar system just like the Kuiper belt.', 'The asteroid belt; short period 

maybe someday implement typed inputs...this caused too much trouble

In [57]:
# from typing import List
# from dspy.functional import TypedPredictor, TypedChainOfThought
# from pydantic import BaseModel, Field

# class Input(BaseModel):
#     question: str = Field(description="The question.")
#     #options: List[str] = Field(description="A list of options.")
#     options: str = Field(description="A list of options.")

# class OutputCode(BaseModel):
#     #answer: str = Field(description="must be one of the first ten letters in the English alphabet: A,B,C,D,E,F,G,H,I,J")
#     answer_index: str = Field(description="must be one of the first ten letters in the English alphabet: A,B,C,D,E,F,G,H,I,J")
#     cot_content: str = Field(description="your full rationale for why you chose the answer")



Make the signature and the module

In [8]:
class MMLUPro(dspy.Signature):
    """Answer questions with a rationale and a multiple choice answer."""

    question = dspy.InputField()
    options = dspy.InputField()
    cot_content = dspy.OutputField(desc="your full rationale for why you chose the answer")
    answer_index = dspy.OutputField(desc="reply with the index of the correct answer from the options. you must reply with only one choice from 0-9")
    



In [9]:
class CoTMMLU(dspy.Module):  # let's define a new module
    def __init__(self):
        super().__init__()
       
        self.signature = MMLUPro
        self.predictor = dspy.ChainOfThought(self.signature)
    
    def forward(self, question, options):
        result = self.predictor(question=question, options=options)
        return dspy.Prediction(answer_index=str(result.answer_index), cot_content=str(result.cot_content))



test just one question to be sure it works

In [13]:
sample_question = try10ex[0]['question']
sample_options = try10ex[0]['options']
sample_answer_index = try10ex[0]['answer_index']

print("Sample Question:", sample_question)
print("Options:", sample_options)
print("Answer Index:", sample_answer_index)




Sample Question: Why are parvoviruses a highly impactful parasite?
Options: ["They are able to alter the host's DNA", 'Because they have no nucleic acid', 'They can survive in extreme temperatures', 'Only replicate in dividing cells', 'They can infect multiple species', "They don't require a host to survive", 'Can integrate into host chromosomes', 'N/A', 'N/A', 'N/A']
Answer Index: 1


In [14]:
uncompiled = CoTMMLU()
response = uncompiled(question=sample_question, options=str(sample_options))
display(response)




Prediction(
    answer_index='3',
    cot_content='Parvoviruses are highly impactful because they can only replicate in dividing cells. This dependency on host cell division means that they can cause significant damage to tissues where cells are rapidly dividing. For example, in dogs, canine parvovirus targets the rapidly dividing cells in the intestines, leading to severe gastrointestinal illness. In humans, parvovirus B19 targets erythroid progenitor cells in the bone marrow, which can lead to conditions like anemia. This specific replication requirement is a key factor in their pathogenicity.'
)

In [ ]:
llm.inspect_history(n=1)

bootstrapfewshot as a simple optimizer...later we can get crazy, just prove the workflow for now.

In [15]:
from dspy.teleprompt import BootstrapFewShot

def custom_answer_exact_match(prediction, example, trace=None):
    return prediction.answer_index == example['answer_index']

compiled = BootstrapFewShot(
    metric=custom_answer_exact_match,
    # num_threads=30,
    # num_candidate_programs=5,
    max_labeled_demos=8,
).compile(
    student=CoTMMLU(),
    trainset=try10ex,
)



  0%|          | 0/10 [00:00<?, ?it/s]

 50%|█████     | 5/10 [00:17<00:17,  3.51s/it]

Bootstrapped 4 full traces after 6 examples in round 0.


In [ ]:
llm.inspect_history(n=1)

get 10 new questions 

In [21]:
import random
try10 = random.sample(validation, 10)
#display(try10)

try10test = [example.with_inputs("question", "options") for example in random.sample(validation, 10)]
for ex in try10test:
    ex['options'] = str(ex['options'])
    ex['answer_index'] = str(ex['answer_index'])
display(try10test)

[Example({'question_id': 25, 'question': 'Predict the number of lines in the EPR spectrum of a solution of 13C-labelled methyl radical (13CH3•), assuming the lines do not overlap.', 'options': "['10', '8', '4', '20', '12', '3', '16', '5', '24', '6']", 'answer': 'B', 'answer_index': '1', 'cot_content': "A: Let's think step by step. The electron paramagnetic resonance spectrum will be split by two forms of interactions. The first is the hyperfine interaction with the 13C (nuclear spin $I = \nrac{1}{2}$) which will split the spectrum into 2 lines. This will be further split into 4 lines by the interaction with three equivalent 1H nuclei. The total number of lines is therefore $2 \\cdot 4 = 8$. The answer is (B).", 'category': 'chemistry', 'src': 'cot_lib-college_chemistry'}) (input_keys={'options', 'question'}),
 Example({'question_id': 64, 'question': 'In terms of Hofstede’s (1980) five cultural dimensions, the United States scores at the top of the scale on:', 'options': "['individualis

evaluate uncompiled vs compiled on the 10 new questions

In [17]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be re-used in your code.
#evaluator = Evaluate(devset=try10test, num_threads=1, display_progress=True, display_table=5)
evaluator = Evaluate(devset=try10test, num_threads=1, display_progress=True)

# Launch evaluation.
print("uncompiled")
evaluator(uncompiled, metric=custom_answer_exact_match)
print("---")
print("compiled")
evaluator(compiled, metric=custom_answer_exact_match)
print("---")



uncompiled


Average Metric: 9 / 10  (90.0): 100%|██████████| 10/10 [00:38<00:00,  3.86s/it]


Average Metric: 9 / 10  (90.0%)
---
compiled


Average Metric: 9 / 10  (90.0): 100%|██████████| 10/10 [00:44<00:00,  4.46s/it]

Average Metric: 9 / 10  (90.0%)
---


now we're going to try to use chris dossman's implementation of self-discover
https://colab.research.google.com/drive/1GkAQKmw1XQgg5UNzzy8OncRe79V6pADB?usp=sharing
https://arxiv.org/pdf/2402.03620.pdf

In [5]:
# Usage example
reasoning_modules = [
    "How could I devise an experiment to help solve that problem?",
    "Make a list of ideas for solving this problem, and apply them one by one to the problem to see if any progress can be made.",
    "How could I measure progress on this problem?",
    "How can I simplify the problem so that it is easier to solve?",
    "What are the key assumptions underlying this problem?",
    "What are the potential risks and drawbacks of each solution?",
    "What are the alternative perspectives or viewpoints on this problem?",
    "What are the long-term implications of this problem and its solutions?",
    "How can I break down this problem into smaller, more manageable parts?",
    "Critical Thinking: This style involves analyzing the problem from different perspectives, questioning assumptions, and evaluating the evidence or information available. It focuses on logical reasoning, evidence-based decision-making, and identifying potential biases or flaws in thinking.",
    "Try creative thinking, generate innovative and out-of-the-box ideas to solve the problem. Explore unconventional solutions, thinking beyond traditional boundaries, and encouraging imagination and originality.",
    "Seek input and collaboration from others to solve the problem. Emphasize teamwork, open communication, and leveraging the diverse perspectives and expertise of a group to come up with effective solutions.",
    "Use systems thinking: Consider the problem as part of a larger system and understanding the interconnectedness of various elements. Focuses on identifying the underlying causes, feedback loops, and interdependencies that influence the problem, and developing holistic solutions that address the system as a whole.",
    "Use Risk Analysis: Evaluate potential risks, uncertainties, and tradeoffs associated with different solutions or approaches to a problem. Emphasize assessing the potential consequences and likelihood of success or failure, and making informed decisions based on a balanced analysis of risks and benefits.",
    "Use Reflective Thinking: Step back from the problem, take the time for introspection and self-reflection. Examine personal biases, assumptions, and mental models that may influence problem-solving, and being open to learning from past experiences to improve future approaches.",
    "What is the core issue or problem that needs to be addressed?",
    "What are the underlying causes or factors contributing to the problem?",
    "Are there any potential solutions or strategies that have been tried before? If yes, what were the outcomes and lessons learned?",
    "What are the potential obstacles or challenges that might arise in solving this problem?",
    "Are there any relevant data or information that can provide insights into the problem? If yes, what data sources are available, and how can they be analyzed?",
    "Are there any stakeholders or individuals who are directly affected by the problem? What are their perspectives and needs?",
    "What resources (financial, human, technological, etc.) are needed to tackle the problem effectively?",
    "How can progress or success in solving the problem be measured or evaluated?",
    "What indicators or metrics can be used?",
    "Is the problem a technical or practical one that requires a specific expertise or skill set? Or is it more of a conceptual or theoretical problem?",
    "Does the problem involve a physical constraint, such as limited resources, infrastructure, or space?",
    "Is the problem related to human behavior, such as a social, cultural, or psychological issue?",
    "Does the problem involve decision-making or planning, where choices need to be made under uncertainty or with competing objectives?",
    "Is the problem an analytical one that requrires data analysis, modeling, or optimization techniques?",
    "Is the problem a design challenge that requires creative solutions and innovation?",
    "Does the problem require addressing systemic or structural issues rather than just individual instances?",
    "Is the problem time-sensitive or urgent, requiring immediate attention and action?",
    "What kinds of solution typically are produced for this kind of problem specification?",
    "Given the problem specification and the current best solution, have a guess about other possible solutions.",
    "Let’s imagine the current best solution is totally wrong, what other ways are there to think about the problem specification?",
    "What is the best way to modify this current best solution, given what you know about these kinds of problem specification?",
    "Ignoring the current best solution, create an entirely new solution to the problem.",
    "Let’s think step by step.",
    "Let’s make a step by step plan and implement it with good notion and explanation."
]
seed_modules_str = '"'+ '","'.join(reasoning_modules)  # for a newline-separated string

# SelectModules
### Purpose

Selects appropriate reasoning modules from a predefined set based on the input task description.

### Parameters

question: The task or question to be solved.

reasoning_modules: A list of available reasoning modules.

### Returns

A dictionary containing selected modules tailored to the task at hand.

In [8]:
import json
from dspy.teleprompt import BootstrapFewShot
# Define some example input-output pairs for module selection
module_selection_examples = [
    dspy.Example(
        task_description="How to improve customer satisfaction for an online store?",
        seed_modules=json.dumps(reasoning_modules),
        selected_modules=json.dumps([
            "Seek input and collaboration from others to solve the problem. Emphasize teamwork, open communication, and leveraging the diverse perspectives and expertise of a group to come up with effective solutions.",
            "Use Reflective Thinking: Step back from the problem, take the time for introspection and self-reflection. Examine personal biases, assumptions, and mental models that may influence problem-solving, and being open to learning from past experiences to improve future approaches.",
            "What are the underlying causes or factors contributing to the problem?",
            "What are the potential obstacles or challenges that might arise in solving this problem?"
        ])
    ).with_inputs('task_description', 'seed_modules'),
    dspy.Example(
        task_description="How to optimize the supply chain for a manufacturing company?",
        seed_modules=json.dumps(reasoning_modules),
        selected_modules=json.dumps([
            "Use systems thinking: Consider the problem as part of a larger system and understanding the interconnectedness of various elements. Focuses on identifying the underlying causes, feedback loops, and interdependencies that influence the problem, and developing holistic solutions that address the system as a whole.",
            "What are the key assumptions underlying this problem?",
            "Is the problem an analytical one that requires data analysis, modeling, or optimization techniques?",
            "Does the problem involve decision-making or planning, where choices need to be made under uncertainty or with competing objectives?"
        ])
    ).with_inputs('task_description', 'seed_modules'),
    dspy.Example(
        task_description="How many numbers between 1 and 2005 are integer multiples of 3 or 4 but not 12?",
        seed_modules=json.dumps(reasoning_modules),
        selected_modules=json.dumps([
            "What is the core issue or problem that needs to be addressed?",
            "How can I measure progress on this problem?",
            "What kinds of solution typically are produced for this kind of problem specification?",
            "Given the problem specification and the current best solution, have a guess about other possible solutions.",
            "Let’s think step by step.",
            "Let’s make a step by step plan and implement it with good notion and explanation."
        ])
    ).with_inputs('task_description', 'seed_modules'),
    dspy.Example(
        task_description="How many numbers are in the list 6,7,10,11,14,15,..., 94,95,98?",
        seed_modules=json.dumps(reasoning_modules),
        selected_modules=json.dumps([
            "What is the core issue or problem that needs to be addressed?",
            "How can I simplify the problem so that it is easier to solve?",
            "How can I measure progress on this problem?",
            "What is the best way to modify this current best solution, given what you know about these kinds of problem specification?",
            "Ignoring the current best solution, create an entirely new solution to the problem.",
            "Let’s think step by step.",
            "Let’s make a step by step plan and implement it with good notion and explanation."
        ])
    ).with_inputs('task_description', 'seed_modules')
]
# Update the signatures to include a mechanism for passing extra context
class SelectModulesSignature(dspy.Signature):
    """"Select several reasoning modules that are crucial to utilize in order to solve the given task, print the complete reasoning module to pass validation"""
    task_description = dspy.InputField(prefix="Task(s) Description:", desc="The task(s) to solve.")
    seed_modules = dspy.InputField(prefix="Relevant Reasoning Modules:", desc="List of relevant reasoning modules to solve task(s) with.")
    selected_modules = dspy.OutputField(prefix="Selected Reasoning Modules (No Hallucinations and write out the whole reasoning module). DO NOT SOLVE THE PROBLEM:", desc="['module_1', 'module_2', ...]")

# Module Selection
class SelectModules(dspy.Module):
    def __init__(self):
        super().__init__()
        self.select_modules = dspy.ChainOfThought(SelectModulesSignature)

    def forward(self, task_description, seed_modules):
        seed_modules_json = json.dumps(seed_modules)
        prediction = self.select_modules(task_description=task_description, seed_modules=seed_modules_json)
        selected_modules = prediction.selected_modules
        # Assert that the selected_modules output is in JSON format

        return dspy.Prediction(selected_modules=selected_modules)


def validate_selected_modules_against_reasoning_modules(example, prediction, something_else):
    # Convert reasoning_modules from JSON string to Python list
    try:
        predicted_selected_modules_list = json.loads(prediction.selected_modules)
    except Exception as e:
        print(f'Failed to load predicted selected modules: {e}')
        return False
    return all(module in reasoning_modules for module in predicted_selected_modules_list)

# Set up a basic teleprompter for module selection
teleprompter_selection = BootstrapFewShot(metric=validate_selected_modules_against_reasoning_modules)

# Compile the SelectModules module
compiled_select_modules = teleprompter_selection.compile(SelectModules(), trainset=module_selection_examples)

 25%|██▌       | 1/4 [00:18<00:56, 18.92s/it]

Failed to load predicted selected modules: Extra data: line 3 column 1 (char 493)


 50%|█████     | 2/4 [00:37<00:37, 18.96s/it]

Failed to load predicted selected modules: Extra data: line 3 column 1 (char 1131)


 75%|███████▌  | 3/4 [00:57<00:19, 19.26s/it]

Failed to load predicted selected modules: Extra data: line 3 column 1 (char 696)


100%|██████████| 4/4 [01:21<00:00, 20.36s/it]

Bootstrapped 0 full traces after 4 examples in round 0.


TODO: solve for "Failed to load predicted selected modules: Extra data: line 3 column 1 (char 696)" with Pydantic typing...

Test for select modules

In [9]:
task_description = 'How many primes are in the row of Pascal’s Triangle that starts with a 1 followed by a 6?'
prediction = compiled_select_modules.forward(task_description=task_description, seed_modules=reasoning_modules)['selected_modules']
prediction

'["What is the core issue or problem that needs to be addressed?", "How can I simplify the problem so that it is easier to solve?", "Critical Thinking: This style involves analyzing the problem from different perspectives, questioning assumptions, and evaluating the evidence or information available. It focuses on logical reasoning, evidence-based decision-making, and identifying potential biases or flaws in thinking.", "Let’s think step by step.", "Let’s make a step by step plan and implement it with good notion and explanation."]\n\n---'

# AdaptModules
### Purpose

Adapts the selected reasoning modules to the specific context and requirements of the task.


### Parameters

selected_modules: The modules selected by the SelectModules component.

question: The task or question to be solved.
### Returns
 
A dictionary of adapted modules ready for implementation in solving the task.

In [10]:
# Define some example input-output pairs for module adaptation
module_adaptation_examples = [
    dspy.Example(
        task_description="How many numbers between 1 and 2005 are integer multiples of 3 or 4 but not 12?",
        selected_modules=json.dumps([
            "What is the core issue or problem that needs to be addressed?",
            "How can I measure progress on this problem?",
            "What kinds of solution typically are produced for this kind of problem specification?",
            "Given the problem specification and the current best solution, have a guess about other possible solutions.",
            "Let’s think step by step.",
            "Let’s make a step by step plan and implement it with good notion and explanation."
        ]),
        adapted_modules=json.dumps([
            "Identify the mathematical pattern or rule that defines the set of numbers.",
            "Determine a method to count the numbers that fit the criteria without enumerating each one.",
            "Consider the inclusion-exclusion principle to avoid overcounting numbers that are multiples of both 3 and 4.",
            "Estimate the number of multiples within the range and adjust for the exclusion of multiples of 12.",
            "Break down the problem into smaller parts: count multiples of 3, multiples of 4, and then exclude multiples of 12.",
            "Implement a systematic approach to calculate the final count, ensuring all criteria are met."
        ])
    ).with_inputs('task_description', 'selected_modules'),
    dspy.Example(
        task_description="A restaurant offers three desserts, and exactly twice as many appetizers as main courses. A dinner consists of an appetizer, a main course, and a dessert. What is the least number of main courses that the restaurant should offer so that a customer could have a different dinner each night in the year 2003?",
        selected_modules=json.dumps([
            "What is the core issue or problem that needs to be addressed?",
            "How can I simplify the problem so that it is easier to solve?",
            "Is the problem an analytical one that requires data analysis, modeling, or optimization techniques?",
            "Let’s think step by step.",
            "Let’s make a step by step plan and implement it with good notion and explanation."
        ]),
        adapted_modules=json.dumps([
            "Determine the total number of dinner combinations needed to cover all nights in the year 2003.",
            "Recognize that the number of dinners is the product of the number of choices for each course.",
            "Formulate the problem as an equation based on the given ratios and the total number of combinations required.",
            "Solve the equation to find the minimum number of main courses needed.",
            "Verify the solution by ensuring that the number of possible dinners exceeds the number of days in the year 2003."
        ])
    ).with_inputs('task_description', 'selected_modules'),
    dspy.Example(
        task_description="How many ways are there to arrange 6 people around a circular table with 7 seats? (Two seatings are considered the same if one is a rotation of the other.)",
        selected_modules=json.dumps([
            "What is the core issue or problem that needs to be addressed?",
            "Is the problem an analytical one that requires data analysis, modeling, or optimization techniques?",
            "Let’s think step by step.",
            "Let’s make a step by step plan and implement it with good notion and explanation."
        ]),
        adapted_modules=json.dumps([
            "Understand the concept of circular permutations and how they differ from linear permutations.",
            "Recognize that one seat can be fixed to account for rotational symmetry, reducing the problem to a linear permutation.",
            "Calculate the number of ways to arrange the remaining people in the available seats.",
            "Consider any constraints or special cases that may affect the arrangement.",
            "Conclude with the total number of distinct circular arrangements for the given scenario."
        ])
    ).with_inputs('task_description', 'selected_modules')
    # Add more examples as needed
]

# Define the signature for module adaptation
class AdaptModulesSignature(dspy.Signature):
    """Rephrase and specify each reasoning module so that it better helps solving the task."""
    selected_modules = dspy.InputField(prefix="Selected Reasoning Modukes:", desc="The selected reasoning modules to use for the task.")
    task_description = dspy.InputField(prefix="Task(s) Description:", desc="The task description.")
    adapted_modules = dspy.OutputField(prefix="Adapted Modules:", desc="['adapted_reasoning_module_1','adapted_reasoning_module_2','adapted_reasoning_module_N']")

# Implement the module for adapting reasoning modules
class AdaptModules(dspy.Module):
    def __init__(self):
        super().__init__()
        self.adapt_modules = dspy.ChainOfThought(AdaptModulesSignature)

    def forward(self, selected_modules, task_description):
        selected_modules_json = json.dumps(selected_modules)
        prediction = self.adapt_modules(selected_modules=selected_modules_json, task_description=task_description)
        adapted_modules = []  # Initialize adapted_modules to an empty list
        try:
            cleaned_output = json.dumps(prediction.adapted_modules.strip())
            adapted_modules = json.loads(cleaned_output)
        except Exception as e:
            print(f'Failed to load adapted modules: {e}')
        return dspy.Prediction(adapted_modules=adapted_modules)

# Validation logic for module adaptation
def validate_adapted_modules(example, pred, trace=None):
    # gold_adapted_modules = json.loads(example.adapted_modules)
    # predicted_adapted_modules = pred.adapted_modules
    return True# set(gold_adapted_modules) == set(predicted_adapted_modules)

# Set up a basic teleprompter for module adaptation
teleprompter_adaptation = BootstrapFewShot(metric=validate_adapted_modules)

# Compile the AdaptModules module
compiled_adapt_modules = teleprompter_adaptation.compile(AdaptModules(), trainset=module_adaptation_examples)

100%|██████████| 3/3 [00:27<00:00,  9.01s/it]

Bootstrapped 3 full traces after 3 examples in round 0.


Test for adapted Modules

In [11]:
task_description = 'How many primes are in the row of Pascal’s Triangle that starts with a 1 followed by a 6?'
selected_modules = ['What is the core issue or problem that needs to be addressed?',
 'Is the problem an analytical one that requires data analysis, modeling, or optimization techniques?',
 'Let’s think step by step.',
 'Let’s make a step by step plan and implement it with good notion and explanation.']

prediction = compiled_adapt_modules.forward(task_description=task_description, selected_modules=selected_modules)
prediction['adapted_modules']

"1. **Identify the specific row in Pascal's Triangle**: Determine which row in Pascal's Triangle starts with 1 followed by 6.\n2. **Calculate the binomial coefficients**: Compute the binomial coefficients for the identified row.\n3. **Check for primality**: Determine which of the binomial coefficients are prime numbers.\n4. **Count the prime numbers**: Count the number of prime numbers in the identified row.\n5. **Verify the results**: Ensure that all steps are correctly followed and the count of prime numbers is accurate."

#ImplementStructure
###Purpose

Translates the adapted modules into a concrete, step-by-step reasoning structure in JSON format.


###Parameters

adapted_modules: Adapted modules from the AdaptModules component.

task_description: Detailed description of the task to be solved.

###Returns

A JSON object detailing the step-by-step reasoning structure for solving the task.

In [14]:
import re
implement_structure_examples = [
    dspy.Example(
        task_description='This SVG path element <path d="M 55.57,80.69 L 57.38,65.80 M 57.38,65.80 L 48.90,57.46 M 48.90,57.46 L 45.58,47.78 M 45.58,47.78 L 53.25,36.07 L 66.29,48.90 L 78.69,61.09 L 55.57,80.69"/> draws a:(A) circle (B) heptagon (C) hexagon (D) kite (E) line (F) octagon (G) pentagon(H) rectangle (I) sector (J) triangle)',
        adapted_modules=json.dumps([
            "Analyze the SVG path commands and coordinates to understand the shape being drawn.",
            "Visualize the sequence of moves and lines to form a mental image of the resulting shape.",
            "Compare the visualized shape with the characteristics of known geometric shapes to identify a match.",
            "Use pattern recognition and knowledge of geometric properties to interpret the SVG path.",
            "Verify the conclusion by sketching or digitally rendering the SVG path to ensure accuracy."
        ]),
        reasoning_structure=json.dumps({
            "steps": [
                {
                    "step": 1,
                    "description": "Examine the SVG path commands and break them down into individual movements and lines.",
                    "action": "Identify each command in the SVG path and understand its function, such as moving to a new starting point or drawing a line."
                },
                {
                    "step": 2,
                    "description": "Analyze the Move to Command (M) and Line to Command (L) within the SVG path.",
                    "action": "Determine the starting points and the end points of each line segment to visualize the shape being drawn."
                },
                {
                    "step": 3,
                    "description": "Visualize the sequence of moves and lines to form a mental image of the resulting shape.",
                    "action": "Use the coordinates provided by the path commands to plot the points and connect them in the order they appear."
                },
                {
                    "step": 4,
                    "description": "Compare the visualized shape with the characteristics of known geometric shapes to identify a match.",
                    "action": "Assess the number of sides, angles, and other geometric properties to classify the shape."
                },
                {
                    "step": 5,
                    "description": "Use pattern recognition and knowledge of geometric properties to interpret the SVG path.",
                    "action": "Apply knowledge of geometric shapes to determine if the path represents a common geometric figure."
                },
                {
                    "step": 6,
                    "description": "Verify the conclusion by sketching or digitally rendering the SVG path to ensure accuracy.",
                    "action": "Create a visual representation of the path to confirm the shape identified in the previous steps."
                }
            ],
            "conclusion": {
                "description": "Synthesize the information from the analysis to determine the geometric shape represented by the SVG path.",
                "action": "Review the visual representation and the properties of the shape to conclude the type of geometric figure it represents."
            }
        })
    ).with_inputs('task_description', 'adapted_modules'),
    dspy.Example(
        task_description='A customer ordered 15 pieces of gourmet chocolate. The order can be packaged in small boxes that contain 1, 2 or 4 pieces of chocolate. Any box that is used must be full. How many different combinations of boxes can be used for the customer’s 15 chocolate pieces? One such combination to be included is to use seven 2-piece boxes and one 1-piece box.',
        adapted_modules=json.dumps([
            "Identify the constraints on box sizes and the requirement for boxes to be full.",
            "Systematically explore all combinations of box sizes that sum up to 15 pieces of chocolate.",
            "Iterate through possible numbers of 4-piece boxes, adjusting for the remaining pieces.",
            "For each number of 4-piece boxes, iterate through possible numbers of 2-piece boxes, adjusting for the remaining pieces.",
            "Determine if the remaining pieces can be filled with 1-piece boxes for each combination of 4-piece and 2-piece boxes.",
            "Ensure all possible combinations are accounted for without duplication."
        ]),
        reasoning_structure=json.dumps({
            "steps": [
                {
                    "step": 1,
                    "description": "Identify the constraints on box sizes and the requirement for boxes to be full.",
                    "action": "Note that boxes can only contain 1, 2, or 4 pieces of chocolate and must be full."
                },
                {
                    "step": 2,
                    "description": "Systematically explore all combinations of box sizes that sum up to 15 pieces of chocolate.",
                    "action": "Start with combinations involving the largest box size and work down to the smallest."
                },
                {
                    "step": 3,
                    "description": "Iterate through possible numbers of 4-piece boxes, adjusting for the remaining pieces.",
                    "action": "Calculate how many pieces are left after using 0, 1, 2, or 3 boxes of 4 pieces."
                },
                {
                    "step": 4,
                    "description": "For each number of 4-piece boxes, iterate through possible numbers of 2-piece boxes, adjusting for the remaining pieces.",
                    "action": "For each scenario from step 3, calculate how many 2-piece boxes can be used with the remaining pieces."
                },
                {
                    "step": 5,
                    "description": "Determine if the remaining pieces can be filled with 1-piece boxes for each combination of 4-piece and 2-piece boxes.",
                    "action": "Check if the remaining pieces after steps 3 and 4 can be exactly filled with 1-piece boxes."
                },
                {
                    "step": 6,
                    "description": "Ensure all possible combinations are accounted for without duplication.",
                    "action": "Review all generated combinations to ensure no duplicates and all possibilities are considered."
                }
            ],
            "conclusion": {
                "description": "Calculate the total number of unique combinations that can be used to package the 15 pieces of chocolate.",
                "action": "Summarize the findings and present the total number of distinct packaging combinations."
            }
        })
    ).with_inputs('task_description', 'adapted_modules')
]


# Define the signature for implementing the reasoning structure
class ImplementStructureSignature(dspy.Signature):
    """Operationalize the reasoning modules into a step-by-step reasoning plan in JSON format. Only output JSON no other text:"""
    adapted_modules = dspy.InputField(prefix="Adapted Modules:", desc="Adapted Modules")
    task_description = dspy.InputField(prefix="Task(s) Description:", desc="The task description.")
    reasoning_structure = dspy.OutputField(prefix="Reasoning Structure JSON:", desc="Implement a reasoning structure for solvers to follow step-by-step and arrive at the correct answers. just json no other text")

# Implement the module for creating a reasoning structure
class ImplementStructure(dspy.Module):
    def __init__(self):
        super().__init__()
        self.implement_structure = dspy.Predict(ImplementStructureSignature)

    def forward(self, adapted_modules, task_description):
        adapted_modules_json = json.dumps(adapted_modules)
        prediction = self.implement_structure(adapted_modules=adapted_modules_json, task_description=task_description)
        reasoning_structure = {}  # Initialize reasoning_structure to an empty dictionary
        try:
            reasoning_structure_output = prediction.reasoning_structure
            reasoning_structure_cleaned = re.sub(r'^\s*```json\n|\n```$', '', reasoning_structure_output, flags=re.MULTILINE).strip()
            reasoning_structure = json.loads(reasoning_structure_cleaned)
        except Exception as e:
            print(f'Failed to load reasoning structure: {e}')
        return dspy.Prediction(reasoning_structure=reasoning_structure)

# Validation logic for module adaptation
def validate_adapted_modules(example, pred, trace=None):

    return True# set(gold_adapted_modules) == set(predicted_adapted_modules)

# Compile the ImplementStructureSignature using the teleprompter
teleprompter_implement_structure = BootstrapFewShot(metric=validate_adapted_modules)
compiled_implement_structure = teleprompter_implement_structure.compile(ImplementStructure(), trainset=implement_structure_examples)

100%|██████████| 2/2 [00:00<00:00, 1546.57it/s]

Bootstrapped 2 full traces after 2 examples in round 0.


In [15]:
# Call the compiled ImplementStructure module with example inputs
example_adapted_modules = '["Identify the specific row in Pascal\'s Triangle starting with 1 followed by a 6.", "Understand the definition of prime numbers and the criteria for identifying them.", "Systematically check each number in the row (excluding the 1s) for primality.", "Tally the numbers confirmed to be prime to answer the question."]'

task_description = 'How many primes are in the row of Pascal’s Triangle that starts with a 1 followed by a 6?'

# Generate the reasoning structure
reasoning_structure_prediction = compiled_implement_structure(adapted_modules=example_adapted_modules, task_description=task_description)

# Print the reasoning structure
print(reasoning_structure_prediction.reasoning_structure)

{'steps': [{'step': 1, 'description': "Identify the specific row in Pascal's Triangle starting with 1 followed by a 6.", 'action': "Recognize that the row in Pascal's Triangle starting with 1 followed by a 6 is the 6th row (0-indexed)."}, {'step': 2, 'description': 'Understand the definition of prime numbers and the criteria for identifying them.', 'action': 'Recall that a prime number is a natural number greater than 1 that has no positive divisors other than 1 and itself.'}, {'step': 3, 'description': "List the elements of the 6th row of Pascal's Triangle.", 'action': "Generate the 6th row of Pascal's Triangle: [1, 6, 15, 20, 15, 6, 1]."}, {'step': 4, 'description': 'Systematically check each number in the row (excluding the 1s) for primality.', 'action': 'Check the numbers 6, 15, 20, 15, and 6 for primality.'}, {'step': 5, 'description': 'Tally the numbers confirmed to be prime to answer the question.', 'action': 'Count the prime numbers identified in the row.'}], 'conclusion': {'de

#Solver
###Purpose

Applies the reasoning structure to generate a solution to the task.


###Parameters

question: The task or question to be solved.

reasoning_structure: The step-by-step plan generated by the ImplementStructure component.
###Returns

The solution to the task as determined through the application of the reasoning structure.

In [16]:
solver_examples = [
    dspy.Example(
        task_description="How many numbers between 1 and 2005 are integer multiples of 3 or 4 but not 12?",
        reasoning_structure=json.dumps({
            "steps": [
                {
                    "step": 1,
                    "description": "Identify the mathematical pattern or rule that defines the set of numbers.",
                    "action": "Determine the multiples of 3 and 4 up to 2005, excluding those that are multiples of 12."
                },
                {
                    "step": 2,
                    "description": "Determine a method to count the numbers that fit the criteria without enumerating each one.",
                    "action": "Use the inclusion-exclusion principle to calculate the count."
                },
                {
                    "step": 3,
                    "description": "Estimate the number of multiples within the range and adjust for the exclusion of multiples of 12.",
                    "action": "Subtract the count of multiples of 12 from the combined count of multiples of 3 and 4."
                },
                {
                    "step": 4,
                    "description": "Break down the problem into smaller parts: count multiples of 3, multiples of 4, and then exclude multiples of 12.",
                    "action": "Calculate the counts separately and then combine them according to the inclusion-exclusion principle."
                },
                {
                    "step": 5,
                    "description": "Implement a systematic approach to calculate the final count, ensuring all criteria are met.",
                    "action": "Verify the calculations and ensure no number is counted more than once."
                }
            ],
            "conclusion": {
                "description": "Synthesize the information from the analysis to determine the final count of numbers.",
                "action": "Combine the results from each step to arrive at the final answer."
            }
        }),
        solution=json.dumps({
            "steps": [
                {
                    "step": 1,
                    "answer": "There are 668 multiples of 3 and 501 multiples of 4 between 1 and 2005."
                },
                {
                    "step": 2,
                    "answer": "Using the inclusion-exclusion principle, we can find the count without listing each number."
                },
                {
                    "step": 3,
                    "answer": "There are 167 multiples of 12 between 1 and 2005."
                },
                {
                    "step": 4,
                    "answer": "The combined count of multiples of 3 and 4 is 1169, but we must exclude the multiples of 12."
                },
                {
                    "step": 5,
                    "answer": "After excluding multiples of 12, we are left with 1002 numbers."
                }
            ],
            "conclusion": {
                "answer": "1002"
            }
        })
    ).with_inputs('task_description', 'reasoning_structure'),
]

# Define the signature for implementing the reasoning structure with answers
class SolverSignature(dspy.Signature):
    """Operationalize the reasoning modules into a step-by-step reasoning plan with answers and a conclusion."""
    task_description = dspy.InputField(prefix="Adapted Modules:", desc="The task description.")
    reasoning_structure = dspy.InputField(prefix="Reasoning Structure:", desc="The reasoning structure")
    solution = dspy.OutputField(prefix="Solution:", desc='The complete reasoning plan with answers and a conclusion in JSON format. example:"{steps[{"step": N,"answer": "ASNWER TEXT"}],"conclusion": {"answer": "ONLY ANSWER NO SUPPORTING TEXT"}}')

# Implement the solver module
class Solver(dspy.Module):
    def __init__(self):
        super().__init__()
        self.solve_task = dspy.Predict(SolverSignature)

    def forward(self, task_description, reasoning_structure):
        reasoning_structure_json = json.dumps(reasoning_structure)
        prediction = self.solve_task(task_description=task_description, reasoning_structure=reasoning_structure_json)
        solution = {}  # Initialize solution to an empty dictionary
        try:
            cleaned_output = re.sub(r'^\s*```json\n|\n```$', '', prediction.solution, flags=re.MULTILINE).strip()
            solution = json.loads(cleaned_output)
        except Exception as e:
            print(f'Failed to load solution: {e}')
        return dspy.Prediction(solution=solution)

# Validation logic for the solver
def validate_solution(example, pred, trace=None):
    # Here you would compare the predicted solution with the expected solution
    # For simplicity, we are returning True
    return True

# Compile the Solver module using the teleprompter
teleprompter_solver = BootstrapFewShot(metric=validate_solution)

# Compile the Solver module
compiled_solver = teleprompter_solver.compile(Solver(), trainset=solver_examples)

100%|██████████| 1/1 [00:15<00:00, 15.80s/it]

Bootstrapped 1 full traces after 1 examples in round 0.


In [18]:
display(compiled_solver)



solve_task = Predict(SolverSignature(task_description, reasoning_structure -> solution
    instructions='Operationalize the reasoning modules into a step-by-step reasoning plan with answers and a conclusion.'
    task_description = Field(annotation=str required=True json_schema_extra={'prefix': 'Adapted Modules:', 'desc': 'The task description.', '__dspy_field_type': 'input'})
    reasoning_structure = Field(annotation=str required=True json_schema_extra={'prefix': 'Reasoning Structure:', 'desc': 'The reasoning structure', '__dspy_field_type': 'input'})
    solution = Field(annotation=str required=True json_schema_extra={'prefix': 'Solution:', 'desc': 'The complete reasoning plan with answers and a conclusion in JSON format. example:"{steps[{"step": N,"answer": "ASNWER TEXT"}],"conclusion": {"answer": "ONLY ANSWER NO SUPPORTING TEXT"}}', '__dspy_field_type': 'output'})
))

In [17]:
# Function to call the compiled_solver with a new novel example input without the answers
def call_solver_with_new_input(task_description, reasoning_structure):
    # Convert the reasoning structure to JSON format for the solver input
    reasoning_structure_json = json.dumps(reasoning_structure)

    # Use the compiled solver to predict the solution for the new example input
    prediction = compiled_solver.solve_task(task_description=task_description, reasoning_structure=reasoning_structure_json)

    # Return the predicted solution
    return prediction.solution

# Example of calling the function with a new input
new_example_input = {
    "task_description": "A customer ordered 15 pieces of gourmet chocolate. The order can be packaged in small boxes that contain 1, 2 or 4 pieces of chocolate. Any box that is used must be full. How many different combinations of boxes can be used for the customer’s 15 chocolate pieces? One such combination to be included is to use seven 2-piece boxes and one 1-piece box.",
    "reasoning_structure": {
        "steps": [
            {
                "step": 1,
                "description": "Identify the constraints on box sizes and the requirement for boxes to be full.",
                "action": "Note that boxes can only contain 1, 2, or 4 pieces of chocolate and must be full."
            },
            {
                "step": 2,
                "description": "Systematically explore all combinations of box sizes that sum up to 15 pieces of chocolate.",
                "action": "Start with combinations involving the largest box size and work down to the smallest."
            },
            {
                "step": 3,
                "description": "Iterate through possible numbers of 4-piece boxes, adjusting for the remaining pieces.",
                "action": "Calculate how many pieces are left after using 0, 1, 2, or 3 boxes of 4 pieces."
            },
            {
                "step": 4,
                "description": "For each number of 4-piece boxes, iterate through possible numbers of 2-piece boxes, adjusting for the remaining pieces.",
                "action": "For each scenario from step 3, calculate how many 2-piece boxes can be used with the remaining pieces."
            },
            {
                "step": 5,
                "description": "Determine if the remaining pieces can be filled with 1-piece boxes for each combination of 4-piece and 2-piece boxes.",
                "action": "Check if the remaining pieces after steps 3 and 4 can be exactly filled with 1-piece boxes."
            },
            {
                "step": 6,
                "description": "Ensure all possible combinations are accounted for without duplication.",
                "action": "Review all generated combinations to ensure no duplicates and all possibilities are considered."
            }
        ],
        "conclusion": {
            "description": "Calculate the total number of unique combinations that can be used to package the 15 pieces of chocolate.",
            "action": "Summarize the findings and present the total number of distinct packaging combinations."
        }
    }
}

# Example call to the function
# Note: In a real scenario, the 'answer' fields would not be included in the input
predicted_solution = call_solver_with_new_input(new_example_input['task_description'], new_example_input['reasoning_structure'])

# Print the predicted solution
print("Predicted Solution:", predicted_solution)

Predicted Solution: ```json
{
  "steps": [
    {
      "step": 1,
      "description": "Identify the constraints on box sizes and the requirement for boxes to be full.",
      "action": "Note that boxes can only contain 1, 2, or 4 pieces of chocolate and must be full.",
      "answer": "Boxes can contain 1, 2, or 4 pieces of chocolate."
    },
    {
      "step": 2,
      "description": "Systematically explore all combinations of box sizes that sum up to 15 pieces of chocolate.",
      "action": "Start with combinations involving the largest box size and work down to the smallest.",
      "answer": "Explore combinations starting with 4-piece boxes, then 2-piece boxes, and finally 1-piece boxes."
    },
    {
      "step": 3,
      "description": "Iterate through possible numbers of 4-piece boxes, adjusting for the remaining pieces.",
      "action": "Calculate how many pieces are left after using 0, 1, 2, or 3 boxes of 4 pieces.",
      "answer": "Possible numbers of 4-piece boxes: 0, 

In [22]:
display(try10test)



[Example({'question_id': 25, 'question': 'Predict the number of lines in the EPR spectrum of a solution of 13C-labelled methyl radical (13CH3•), assuming the lines do not overlap.', 'options': "['10', '8', '4', '20', '12', '3', '16', '5', '24', '6']", 'answer': 'B', 'answer_index': '1', 'cot_content': "A: Let's think step by step. The electron paramagnetic resonance spectrum will be split by two forms of interactions. The first is the hyperfine interaction with the 13C (nuclear spin $I = \nrac{1}{2}$) which will split the spectrum into 2 lines. This will be further split into 4 lines by the interaction with three equivalent 1H nuclei. The total number of lines is therefore $2 \\cdot 4 = 8$. The answer is (B).", 'category': 'chemistry', 'src': 'cot_lib-college_chemistry'}) (input_keys={'options', 'question'}),
 Example({'question_id': 64, 'question': 'In terms of Hofstede’s (1980) five cultural dimensions, the United States scores at the top of the scale on:', 'options': "['individualis

In [24]:
from dspy.evaluate import Evaluate



# def eval_metric(true, prediction, trace=None):
#     true_answer = true.answer
#     reasoning_structure = str(prediction['steps'])
#     predicted_answer = prediction['conclusion']['answer']

#     is_correct_pred = compiled_answer_evaluator(
#         question=true.question, reasoning_structure=reasoning_structure,predicted_answer=predicted_answer, gold_answer=true_answer
#     )
#     if is_correct_pred.is_correct.lower() == "true":
#         return True
#     else:
#         print(f'evaluation failed for {true.question}:')
#         print(f'{predicted_answer} against {true_answer}')
#         return False

class CombinedModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.select_modules = SelectModules()
        self.adapt_modules = AdaptModules()
        self.implement_structure = ImplementStructure()
        self.solver = Solver()

    def forward(self, question):
        selected_modules = self.select_modules(question, reasoning_modules).selected_modules
        adapted_modules = self.adapt_modules(selected_modules, question).adapted_modules
        reasoning_structure = self.implement_structure(adapted_modules, question).reasoning_structure
        solution = self.solver(question, reasoning_structure).solution
        return solution

combined_module = CombinedModule()

try1 = combined_module(try10test[0].question)
display(try1)

# Evaluation code (same as before)
# evaluate = Evaluate(devset=try10test, metric=custom_answer_exact_match, num_threads=25, display_progress=True, display_table=10)
# evaluate(combined_module)

{'steps': [{'step': 1,
   'answer': 'The key assumptions are that the methyl radical is 13C-labelled (13CH3•) and that the EPR lines do not overlap.'},
  {'step': 2,
   'answer': 'The problem can be broken down into understanding the hyperfine interactions in the EPR spectrum, specifically the interactions between the unpaired electron and the nuclei (13C and 3 H).'},
  {'step': 3,
   'answer': 'Analyzing the problem, we know that 13C has a nuclear spin (I) of 1/2 and each hydrogen (H) has a nuclear spin (I) of 1/2. The hyperfine interaction will split the EPR signal based on these nuclear spins.'},
  {'step': 4,
   'answer': 'The underlying factors include the number of interacting nuclei and their respective nuclear spins. For 13CH3•, there is one 13C nucleus and three 1H nuclei.'},
  {'step': 5,
   'answer': 'Potential obstacles include the complexity of the hyperfine splitting patterns and ensuring that all possible interactions are considered.'},
  {'step': 6,
   'answer': 'The ri

future directions:

does retrieving wikipedia colbert give better mmlupro results? a la https://github.com/stanfordnlp/dspy/blob/5db3e3461a5db8ddb0a03c8ae59e606f4efc1e95/intro.ipynb#L4

score is number of tries to get the correct answer? or just exact match boolean 0 / 1?

does training on mmlu give a smarter bot? need to make options optional. 

kind of like - are people that are really good at taking tests actually useful? are bots that are really good at benchmarks useful?

proper optimizer copro / mipro, work up to using the full 70 in the validation set